# TME 4 Souleymane Mbaye

Les tmes de IAMSI sont très interrestants. Certaines des fonctions que j'ai écrites sont optimisables. Et il y'a des clauses qui occurent plusieurs fois. J'aurais aimé les mettre à niveau mais il y'a beaucoup de choses à faire. Une première version qui fonctionne très très bien c'est l'essentiel je pense.

## Exercice 1: Prise en main de glucose et DIMACS

La traduction de la formule F de l'exercie 2 est dans le fichier exercice2-Mbaye.cnf.
bin/glucose_static tme4-Mbaye/exercice2.cnf nous rend bien s UNSATISFIABLE.

La traduction de la première formule de l'exercice 4 du Td se trouve dans le fichier exercice4-1-Mbaye.cnf.
bin/glucose_static -model tme4-Mbaye/exercice4_1-Mbaye.cnf nous rend:
s SATISFIABLE
v 1 -2 -3 4 -5 -6 7 -8 0. 
La formule est satisfiable avec {x1,-x2,-x3,x4,-x5,-x6,x7,-x8}.

La traduction de la deuxième formule de l'exercice 4 du Td se trouve dans le fichier exercice4-2-Mbaye.cnf.
bin/glucose_static -model tme4-Mbaye/exercice4-2-Mbaye.cnf nous rend:
s SATISFIABLE
v 1 -2 -3 4 5 -6 -7 -8 -9 10 -11 12 -13 -14 -15 -16 0
La formule est satisfiable avec {x1, -x2, -x3, x4, x5, -x6, -x7, -x8, -x9, x10, -x11, x12, -x13, -x14, -15, -x16}.

## Exercice 2: Modélisation

### Question 1

Nous avons m<sub>jxy</sub> la variable qui représente le match du jour j de l'équipe x jouant à domicile contre l'équipe y. Donc nous avons une combinaison de $nj*ne*ne$ = $nj*ne$<sup>2</sup> varibles propositionnelles possibles.

### Question 2

In [6]:
def codage(ne,nj,j,x,y):
  if (nj <= j) or (ne <= x) or (ne <= y):
    return -1
  return j * (ne**2) + x*ne + y + 1

### Question 3

In [49]:
def decodage(k,ne):
  k -= 1  # k = j * (ne**2) + x*ne + y = ne(j*ne+x) + y
  y = k % ne
  k -= y  # k = = ne(j*ne+x)
  k = k // ne # k = j*ne+x
  x = k % ne
  j = k // ne
  
  return j,x,y

In [3]:
ne = 20
nj = ne * (ne-1) # chaque équipe accueille toutes les autres (si au pire un match par jour)
j = 13
x = 4
y = 17

k = codage(ne,nj,j,x,y)
decodage(k,ne)

(13, 4, 17)

## Exercice 3: Génération d'un planning de match

### Question 1. Contrainte de cardinalité

In [18]:
def clauses_format_dimacs(contraintes):
  """On reçoit une liste des clauses et on transforme les nombres en chaine de caractères et on ajoute le zéro de fin de ligne

  Args:
      contraintes (list[list]): liste de liste d'entiers, chaque sous liste est une clause représentée par ses entiers le composant

  Returns:
      str: retourne une chaine de caractères représentant toutes les contraintes de tous les clauses dans le format DIMACS
  """
  contraintes_s = []
  for cont in contraintes:
    cont_s = [str(x) for x in cont]
    cont_s = " ".join(cont_s) + " 0"
    contraintes_s.append(cont_s)
  
  return "\n".join(contraintes_s)

def generer_fichier_dimacs(ne,nj,clauses,nom_fic):
  with open(nom_fic,'w') as f:
    f.write('c Planning match\n')
    f.write(f"p cnf {nj*ne*ne} {len(clauses)}\n")
    f.write(clauses_format_dimacs(clauses))
  

In [2]:
def au_moins(l):
  return  [[x for x in l]]  # rend une liste de clauses (ici un seul élément dans la liste)


def au_plus(l):
  # rend la liste des clauses (sans le zéro à la fin, on l'ajoutera quand on convertira en chaine de caractère)
  clauses = []
  for i in range(len(l)-1):
    x = l[i]
    for y in l[1+i:]:
      clauses.append([-x,-y])
  
  return clauses

In [3]:
l = [1,2,3,4,5]
print(clauses_format_dimacs(au_moins(l)))
print(clauses_format_dimacs(au_plus(l)))

1 2 3 4 5 0
-1 -2 0
-1 -3 0
-1 -4 0
-1 -5 0
-2 -3 0
-2 -4 0
-2 -5 0
-3 -4 0
-3 -5 0
-4 -5 0


### Question 2. Traduction du problème

#### 1. Chaque équipe ne peut jouer plus d'un match par jour

Pour toute équipe x et un jour j nous avons $\sum\limits_{y=1}^{ne} m_{j,x,y} + m_{j,y,x} <= 1$. Une équipe ne peut contre elle même non plus.

In [23]:
def equipe_match_par_jour(ne,nj,j,x):
  # tous matchs possible avec x le jour j
  l = [codage(ne,nj,j,x,y) for y in range(ne) if y!=x] + [codage(ne,nj,j,y,x) for y in range(ne) if y!=x]

  return [[-codage(ne,nj,j,x,x)]] + au_plus(l)  # une équipe ne peut jouer contre elle même

In [7]:
ne = 3 # Par exemple
nj = ne*(ne-1)
x = 1
j = 2
print(clauses_format_dimacs(equipe_match_par_jour(ne,nj,j,x)))

-23 0
-22 -24 0
-22 -20 0
-22 -26 0
-24 -20 0
-24 -26 0
-20 -26 0


#### 2. Fonction encoderC1

Pour toute équipe x et un jour j nous avons $\sum\limits_{y=1}^{ne} m_{j,x,y} + m_{j,y,x} <= 1$. Une équipe ne peut contre elle même non plus.

In [10]:
def encoderC1(ne,nj):
  clauses = []
  for j in range(nj):
    for x in range(ne):
      clauses.extend(equipe_match_par_jour(ne,nj,j,x))
      
  return clauses

#### 3. Nombre de contraintes et de clauses pour 3 équipes sur 4 jours

In [11]:
ne = 3
nj = 4
conts = encoderC1(ne,nj)
print('Nb clauses:',len(conts))
print(clauses_format_dimacs(conts))

Nb clauses: 84
-1 0
-2 -3 0
-2 -4 0
-2 -7 0
-3 -4 0
-3 -7 0
-4 -7 0
-5 0
-4 -6 0
-4 -2 0
-4 -8 0
-6 -2 0
-6 -8 0
-2 -8 0
-9 0
-7 -8 0
-7 -3 0
-7 -6 0
-8 -3 0
-8 -6 0
-3 -6 0
-10 0
-11 -12 0
-11 -13 0
-11 -16 0
-12 -13 0
-12 -16 0
-13 -16 0
-14 0
-13 -15 0
-13 -11 0
-13 -17 0
-15 -11 0
-15 -17 0
-11 -17 0
-18 0
-16 -17 0
-16 -12 0
-16 -15 0
-17 -12 0
-17 -15 0
-12 -15 0
-19 0
-20 -21 0
-20 -22 0
-20 -25 0
-21 -22 0
-21 -25 0
-22 -25 0
-23 0
-22 -24 0
-22 -20 0
-22 -26 0
-24 -20 0
-24 -26 0
-20 -26 0
-27 0
-25 -26 0
-25 -21 0
-25 -24 0
-26 -21 0
-26 -24 0
-21 -24 0
-28 0
-29 -30 0
-29 -31 0
-29 -34 0
-30 -31 0
-30 -34 0
-31 -34 0
-32 0
-31 -33 0
-31 -29 0
-31 -35 0
-33 -29 0
-33 -35 0
-29 -35 0
-36 0
-34 -35 0
-34 -30 0
-34 -33 0
-35 -30 0
-35 -33 0
-30 -33 0


Sur chaque jour de match pour chaque équipe une contrainte. nb<sub>cont</sub>=$4*3$=12. Chaque contrainte est traduite par $C_{2*(ne-1)}^2+1$ clauses. Donc Nb<sub>clauses</sub>$=12*(C_{4}^2+1) = 12*(6+1) = 12*7 = 84$. Mais dans ce cas il y'a beaucoup de répétition de clauses.

#### 4. Chaque équipe doit rencontrer l'ensemble des autres équipes, une fois à domicile et une fois à l'extérieur.

Pour tout couple d'équipe x et y, x!=y nous avons pas plus d'un match à domicile de chaque coté $\sum\limits_{j=1}^{nj} m_{j,x,y} <= 1$ ; $\sum\limits_{j=1}^{nj} m_{j,y,x} <= 1$.
</br>Et au moins un match chacun sur son terrain $\sum\limits_{j=1}^{nj} m_{j,x,y} >= 1$ ; $\sum\limits_{j=1}^{nj} m_{j,y,x} >= 1$

#### 5. Fonction encoderC2

In [16]:
def encoderC2(ne,nj):
  clauses = []
  # pour chaque couple d'équipe x!=y
  for x in range(ne):
    for y in range(x+1,ne):
      lx = [codage(ne,nj,j,x,y) for j in range(nj)] # sur le terrain de x
      ly = [codage(ne,nj,j,y,x) for j in range(nj)] # sur le terrain de y

      # au moins un match sur le terrain de x et y aussi
      au_m_x = au_moins(lx)
      au_m_y = au_moins(ly)

      # au plus un match sur chaque terrain
      au_p_x = au_plus(lx)
      au_p_y = au_plus(ly)

      # on les ajoute dans l'ensemble des clauses
      clauses.extend(au_m_x)
      clauses.extend(au_m_y)
      clauses.extend(au_p_x)
      clauses.extend(au_p_y)
  
  return clauses

#### 6. Nombre de contraintes et de clauses pour 3 équipes sur 4 jours.

In [79]:
ne = 3
nj = 4
conts = encoderC2(ne,nj)
print('Nb clauses:',len(conts))
print(conts)

Nb clauses: 42
[[2, 11, 20, 29], [4, 13, 22, 31], [-2, -11], [-2, -20], [-2, -29], [-11, -20], [-11, -29], [-20, -29], [-4, -13], [-4, -22], [-4, -31], [-13, -22], [-13, -31], [-22, -31], [3, 12, 21, 30], [7, 16, 25, 34], [-3, -12], [-3, -21], [-3, -30], [-12, -21], [-12, -30], [-21, -30], [-7, -16], [-7, -25], [-7, -34], [-16, -25], [-16, -34], [-25, -34], [6, 15, 24, 33], [8, 17, 26, 35], [-6, -15], [-6, -24], [-6, -33], [-15, -24], [-15, -33], [-24, -33], [-8, -17], [-8, -26], [-8, -35], [-17, -26], [-17, -35], [-26, -35]]


Pour chaque couple d'équipe, combinaison de 2 parmi 3, $C_{3}^2 = 3$ cas. Pour chacun 4 contraintes donc $3*4=12$ contraintes.
Les contraintes au moins 1 sont convertis en une seule clause ou. Et les contraintes au plus un par une combinaison de 2 parmi nj clauses donc Nb<sub>clauses</sub> $=3*(2+2*C_{4}^2)=3*14$ = 42 clauses.

#### 7. Fonction encoder qui encode les contraintes de C1 et C2

In [28]:
def encoder(ne,nj):
  return encoderC1(ne,nj) + encoderC2(ne,nj)

### Question 3. Utilisation du solveur

In [48]:
ne = 3
nj = 6
clauses = encoder(ne,nj)
generer_fichier_dimacs(ne,nj,clauses,"match-championnat-ne3-nj4.cnf")

In [34]:
lk = [4,17,20,33,43,48]
for k in lk:
  print(decodage(k,ne))

(0, 1, 0)
(1, 2, 1)
(2, 0, 1)
(3, 1, 2)
(4, 2, 0)
(5, 0, 2)


Pour mon cas va à merveille. On ajoute rien du tout. Cependant il faut que le nombre de jours soit au moins égal à 6. Trois équipes, du coup pas plus d'un match par jour.

### Question 4. Fonction de décodage du résultat de planning

In [122]:
def decoder_fichier(ne,fic_glucose,fic_noms_eq=False,fic_planning=False):
  # traduction du résultat
  lm = []
  with open(fic_glucose,"r") as f:
    l = f.readline()
    lk = l.split(" ")
    for k in lk:
      k = int(k)
      if k > 0:
        lm.append(decodage(k,ne))
  
  noms = []
  # lecture des noms des équipes
  if fic_noms_eq != False:
    with open(fic_noms_eq,"r") as f:
      noms = f.readlines()
    for i in range(len(noms)):
      noms[i] = noms[i][:-1]
  else:
    # si pas de noms d'équipes donnés(noms par défaut)
    noms = [f"equipe {i}" for i in range(1,1+ne)]
  
  # ecriture dans le fichier de planning des matchs du championnat
  if fic_planning != False:
    with open(fic_planning,"w") as f:
      f.write("Planning des matchs du championnat:\n")
      j0 = -1
      for j,x,y in lm:
        if j != j0:
          j0 = j
          f.write(f"\nJournée {j+1} :\n")
        
        f.write(f"\t{noms[x]} vs {noms[y]}\n")
  else:
    # Affichage si pas de noms de fichier de sortie
    print("Planning des matchs du championnat:\n\n")
    j0 = -1
    for j,x,y in lm:
      if j != j0:
        j0 = j
        print(f"\nJournée {j+1} :\n")
      
      print(f"\t{noms[x]} vs {noms[y]}\n")

In [43]:
decoder_fichier(ne,"sortie.txt","noms-equipes.txt","planning.txt")

### Question 5. Script python

J'ai écrit le script python planning-Mbaye.py dans le dossier tme4-Mbaye. Ce dossier se trouve dans le répertoire glucose-4.2.1. J'ai placé l'exécutable glucose_static dans le dossier bin.</br>
Pour lancer le script, se placer dans le dossier glucose-4.2.1.</br>
Trois cas d'usage possible.</br>
Usage 1: python3 tme4-Mbaye/planning-Mbaye.py _nombre-d'équipes_ _nombre-de-jours_</br>
Usage 2: python3 tme4-Mbaye/planning-Mbaye.py _nombre-d'équipes_ _nombre-de-jours_ _nom-fichier-noms-équipes_</br>
Usage 3: python3 tme4-Mbaye/planning-Mbaye.py _nombre-d'équipes_ _nombre-de-jours_ _nom-fichier-noms-équipes_  _nom-fichier-planning_

## Exercice 4. Nombre de jours minimal

Si le nombre d'équipe ne est pair alors pour une journée toutes les équipes peuvent jouer un match. Ainsi pour jouer contre toutes les autres équipes aller et retour il faut $2*(ne-1)$ jours. C'est suffisant !</br>
Sinon si ne impair alors seules $ne-1$ équipes peuvent jouer pendant un jour. Il faudrait 2 jours pour que chaque équipe ait joué au moins un match. Les $ne-1$ équipes jouent des journées complètes en $2*(ne-2)$ jours et l'équipe restante $2*(ne-1)$ jours de match avec les $ne-1$. Ainsi faudrait $2*(ne-2)+2*(ne-1)=4*ne-6$ jours au moins.

## Exercice 5. Extension : équilibrer les déplacements et les week-ends

### Question 1. Traduction en contraintes de cardinalité

Il faudrait que tout (k+1)uplet de variables il y ait au moins un littéral qui soit évalué à faux.

#### Matchs le dimanche

(a) Chaque équipe joue au moins p<sub>ext</sub>% de ses matchs à l'extérieur le dimanche. $\sum\limits_{j=1}^{nj} \sum\limits_{y=1}^{ne} m_{j,y,x} >= p*nj/2$. avec j%2=1 pour les jours de dimanche, jours impairs.

(b) Chaque équipe joue au moins p<sub>dom</sub>% de ses matchs à domicile le dimanche. $\sum\limits_{j=1}^{nj} \sum\limits_{y=1}^{ne} m_{j,x,y} >= p*nj/2$. avec j%2=1 pour les jours de dimanche, jours impairs.

#### Matchs le consécutifs

(a) Aucune équipe ne joue (strictement) plus de deux matchs consécutifs à l'extérieur. Tout triplet de trois mactchs consécutifs, trois jours consécutifs, il faut au moins un à domicile.

(b) Aucune équipe ne joue (strictement) plus de deux matchs consécutifs à domicile. Tout triplet de trois mactchs consécutifs, trois jours consécutifs, il faut au moins un à domicile.

### Question 2. Encodage des contraintes de cardinalités au plus k

In [96]:
def generer_combinaisons_rec(l, i, lse, out=[]):

  # si tous les éléments sont traités, on ajoute le sous-ensemble courant dans la liste des sous-ensembles
  if i < 0:
    lse.append(out.copy())
    return

  # inclut l'élément courant dans le sous-ensemble courant et se reproduit
  out.append(l[i])
  generer_combinaisons_rec(l, i - 1, lse, out)

  # backtrack : exclure l'élément courant du sous-ensemble courant
  out.pop()

  # supprimer les éléments en double adjacents
  while i > 0 and l[i] == l[i - 1]:
    i = i - 1

  # exclure l'élément courant du sous-ensemble courant et se reproduire
  generer_combinaisons_rec(l, i - 1, lse, out)

def generer_combinaisons(l):
  lse = []
  l.sort()
  generer_combinaisons_rec(l, len(l)-1,lse)
  
  return lse

In [105]:
def au_plus_k(l,k):
  l.sort()
  l = [-x for x in l]
  lse = generer_combinaisons(l)
  clauses = [se for se in lse if len(se)==(k+1)]
  
  return clauses

def au_moins_k(l,k):
  l = [-x for x in l]
  return au_plus_k(l,len(l)-k)

In [98]:
l = [1,2,3,4]
k = 2
au_plus_k(l,k)

[[-1, -2, -3], [-1, -2, -4], [-1, -3, -4], [-2, -3, -4]]

In [99]:
l = [1,2,3,4]
k = 3
au_plus_k(l,k)

[[-1, -2, -3, -4]]

In [100]:
l = [1,2,3,4]
k = 1
au_plus_k(l,k)

[[-1, -2], [-1, -3], [-1, -4], [-2, -3], [-2, -4], [-3, -4]]

### Question 3. Encodage du problème étendu

#### Matchs le dimanche

In [111]:
def exterieur_dimanche(ne,nj,p=0.5):
  k = int(nj*p/2) # nombre de match à l'exterieur au moins
  clauses = []
  j_dimanche = [j for j in range(nj) if j%2==1]
  for x in range(ne):
    lx = [codage(ne,nj,j,y,x) for j in j_dimanche for y in range(ne) if y!=x]
    clauses_x = au_moins_k(lx,k)
    clauses.extend(clauses_x)
  
  return clauses

def domicile_dimanche(ne,nj,p=0.4):
  k = int(nj*p/2) # nombre de match à domicile au moins
  clauses = []
  j_dimanche = [j for j in range(nj) if j%2==1]
  for x in range(ne):
    lx = [codage(ne,nj,j,x,y) for j in j_dimanche for y in range(ne) if y!=x]
    clauses_x = au_moins_k(lx,k)
    clauses.extend(clauses_x)
  
  return clauses

In [117]:
exterieur_dimanche(3,6)

[[52, 49, 34, 31, 16, 13], [53, 47, 35, 29, 17, 11], [51, 48, 33, 30, 15, 12]]

In [116]:
domicile_dimanche(3,6)

[[48, 47, 30, 29, 12, 11], [51, 49, 33, 31, 15, 13], [53, 52, 35, 34, 17, 16]]

#### Matchs le consécutifs

In [113]:
def exterieur_consecutifs(ne,nj):
  clauses = []
  for x in range(ne):
    for j in range(nj-3):
      # matchs de trois jours consécutifs avec x
      clauses_x = [[-codage(ne,nj,j,y1,x),-codage(ne,nj,j+1,y2,x),-codage(ne,nj,j+2,y3,x)] for y1 in range(ne) for y2 in range(ne) for y3 in range(ne)]
      clauses.extend(clauses_x)
  
  return clauses

def domicile_consecutifs(ne,nj):
  clauses = []
  for x in range(ne):
    for j in range(nj-3):
      # matchs de trois jours consécutifs avec x
      clauses_x = [[-codage(ne,nj,j,x,y1),-codage(ne,nj,j+1,x,y2),-codage(ne,nj,j+2,x,y3)] for y1 in range(ne) for y2 in range(ne) for y3 in range(ne)]
      clauses.extend(clauses_x)
  
  return clauses

In [114]:
exterieur_consecutifs(3,4)

[[-1, -10, -19],
 [-1, -10, -22],
 [-1, -10, -25],
 [-1, -13, -19],
 [-1, -13, -22],
 [-1, -13, -25],
 [-1, -16, -19],
 [-1, -16, -22],
 [-1, -16, -25],
 [-4, -10, -19],
 [-4, -10, -22],
 [-4, -10, -25],
 [-4, -13, -19],
 [-4, -13, -22],
 [-4, -13, -25],
 [-4, -16, -19],
 [-4, -16, -22],
 [-4, -16, -25],
 [-7, -10, -19],
 [-7, -10, -22],
 [-7, -10, -25],
 [-7, -13, -19],
 [-7, -13, -22],
 [-7, -13, -25],
 [-7, -16, -19],
 [-7, -16, -22],
 [-7, -16, -25],
 [-2, -11, -20],
 [-2, -11, -23],
 [-2, -11, -26],
 [-2, -14, -20],
 [-2, -14, -23],
 [-2, -14, -26],
 [-2, -17, -20],
 [-2, -17, -23],
 [-2, -17, -26],
 [-5, -11, -20],
 [-5, -11, -23],
 [-5, -11, -26],
 [-5, -14, -20],
 [-5, -14, -23],
 [-5, -14, -26],
 [-5, -17, -20],
 [-5, -17, -23],
 [-5, -17, -26],
 [-8, -11, -20],
 [-8, -11, -23],
 [-8, -11, -26],
 [-8, -14, -20],
 [-8, -14, -23],
 [-8, -14, -26],
 [-8, -17, -20],
 [-8, -17, -23],
 [-8, -17, -26],
 [-3, -12, -21],
 [-3, -12, -24],
 [-3, -12, -27],
 [-3, -15, -21],
 [-3, -15, -24

In [118]:
def encoder_extension(ne,nj):
  return encoder(ne,nj)+\
    exterieur_dimanche(ne,nj)+\
    domicile_dimanche(ne,nj)+\
    exterieur_consecutifs(ne,nj)

In [119]:
ne = 3
nj = 6
clauses = encoder(ne,nj)
generer_fichier_dimacs(ne,nj,clauses,f"log/match-championnat-extension-ne-{ne}-nj-{nj}.cnf")

In [125]:
decoder_fichier(ne,"log/sortie.txt","noms-equipes.txt")

Planning des matchs du championnat:



Journée 1 :

	Marseille vs Paris Saint-Germain


Journée 2 :

	Lyon vs Marseille


Journée 3 :

	Paris Saint-Germain vs Marseille


Journée 4 :

	Marseille vs Lyon


Journée 5 :

	Lyon vs Paris Saint-Germain


Journée 6 :

	Paris Saint-Germain vs Lyon

